In [1]:
# @title Golden-Bigit → Constants: Colab "Press-Go" Test Rig
# @markdown **How to use:** Just run this cell. Tweak the CONFIG block if you want more depth/bases/cycles.
# @markdown Results CSVs will be saved in `/content/` and short summaries will print.

import math, random, statistics, time, os, sys
from collections import Counter, defaultdict
from dataclasses import dataclass
from typing import List, Dict, Tuple, Iterable

import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
import mpmath as mp

# ===============================
# CONFIG — tweak as you like
# ===============================
CFG = dict(
    # Bigit: how many Fibonacci digits (decimal digits) to embed.
    # Larger = stronger alpha test; 8k–20k is fine in Colab; 20k may take longer.
    BIGIT_DECIMAL_DIGITS = 8000,

    # Constants precision (mpmath)
    MP_DPS = 200,  # decimal digits precision for constants/ops

    # Alpha-bridge scan: check D_F(N) * (p/q) vs alpha for q in this set (and p=1..q)
    SCAN_QS = [100, 250, 500, 1000],  # enlarge if you want
    PREFIX_NS = [100, 1000, 5000],    # D_F prefixes to test alpha-bridge at (add 20000 if you like)

    # Automaton schedule(s) (3-beat cycles); choose from: mul3, div3, invert, one_minus, frac (keep [0,1))
    CYCLES = [
        ["mul3", "mul3", "invert"],           # your suggested canon
        ["mul3", "one_minus", "invert"],      # variant
    ],
    AUTOMATON_STEPS = 200,   # steps per schedule

    # Bases & depths for tilt/mix/stride
    BASES = list(range(2,13)),  # 2..12
    DEPTHS = [1000, 2000, 5000],  # digits per base measurement

    # Inversion tiers (apply 1/x then keep fractional part)
    INVERSION_TIERS = [0, 1, 2, 3],  # 0 = raw x, 1 = CF1(x), etc.

    # Controls: extra seeds
    DO_CONTROLS = True,
    CONTROL_RANDOM_SEEDS = 3,      # how many random 0/1 decimals (same length as bigit)
    CONTROL_OTHER_MORPHICS = True, # includes Thue–Morse-like 0/1
    # Output dir
    OUTDIR = "/content",
)

# ===============================
# Utilities
# ===============================

def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)

def now():
    return time.strftime('%Y-%m-%d %H:%M:%S')

# normalize entropy to [0,1]
def entropy_norm_from_counts(counts: List[int], base: int) -> float:
    N = sum(counts)
    if N <= 0: return 0.0
    H = 0.0
    for c in counts:
        if c > 0:
            p = c / N
            H -= p * math.log(p)
    return H / math.log(base)

def longest_run(lst: Iterable[int]) -> int:
    best, cur = 0, 0
    prev = None
    for x in lst:
        if x == prev: cur += 1
        else: cur = 1; prev = x
        if cur > best: best = cur
    return best

def ci95_halfwidth_uniform(base: int, N: int) -> float:
    # For a single digit frequency under p=1/base
    p0 = 1.0/base
    return 1.96 * math.sqrt(p0*(1-p0)/max(N,1))

# ===============================
# Fibonacci word & other sources
# ===============================

def fib_word(n_digits: int) -> str:
    # 0->01, 1->0
    s = "0"
    while len(s) < n_digits:
        s = "".join("01" if ch=="0" else "0" for ch in s)
    return s[:n_digits]

def thue_morse(n_digits: int) -> str:
    # 0->01, 1->10 but using parity of bitcount
    # Build quickly using parity; produce 0/1 characters
    out = []
    x = 0
    while len(out) < n_digits:
        # produce next block
        block_len = min(n_digits - len(out), 4096)
        for i in range(block_len):
            n = len(out) + i
            out.append('1' if bin(n).count('1') % 2 else '0')
    return "".join(out[:n_digits])

def random_01(n_digits: int, p=0.5) -> str:
    return "".join('1' if random.random() < p else '0' for _ in range(n_digits))

def embed_decimal_bigit(bits: str) -> Decimal:
    # D_F = 0.bits (decimal)
    getcontext().prec = max(50, len(bits)//2)
    ten = Decimal(10)
    w = Decimal(0)
    place = Decimal(1) / ten  # 10^-1
    for ch in bits:
        if ch == '1':
            w += place
        place /= ten
    return +w  # apply precision

# ===============================
# Base-digit engine (stretch & wrap) using Decimal for stability
# ===============================

def base_digits_decimal(x: Decimal, base: int, N: int) -> List[int]:
    getcontext().prec = max(50, int(N*math.log(base,10)) + 50)
    out, frac = [], x - int(x)
    b = Decimal(base)
    for _ in range(N):
        frac *= b
        d = int(frac)  # 0..base-1
        out.append(int(d))
        frac -= d
        if frac == 0:
            # Exact termination (unlikely for irrational looking)
            out.extend([0]*(N-len(out)))
            break
    return out

# ===============================
# Inversion tiers (continued fraction "invert & shift")
# ===============================

def cf_invert_shift(x: Decimal) -> Decimal:
    if x == 0: return Decimal(0)
    y = (Decimal(1) / x)
    return y - int(y)

def apply_inversion_tiers(x: Decimal, k: int) -> Decimal:
    y = +x
    for _ in range(k):
        y = cf_invert_shift(y)
    return +y

# ===============================
# Automaton: operations & cycles
# ===============================

def frac_part(x: Decimal) -> Decimal:
    return x - int(x)

def apply_op(x: Decimal, op: str) -> Decimal:
    if op == "mul3": return x * 3
    if op == "div3": return x / 3
    if op == "invert":
        if x == 0: return x
        return Decimal(1) / x
    if op == "one_minus": return 1 - x
    if op == "frac": return frac_part(x)
    raise ValueError(f"Unknown op: {op}")

def run_cycle(x0: Decimal, cycle: List[str], steps: int, keep_frac=True) -> List[Decimal]:
    xs = [+x0]
    x = +x0
    for t in range(1, steps+1):
        op = cycle[(t-1) % len(cycle)]
        x = apply_op(x, op)
        if keep_frac: x = frac_part(x)
        xs.append(+x)
    return xs

# ===============================
# Targets / constants
# ===============================

def build_targets(mp_dps=200) -> Dict[str, Decimal]:
    mp.mp.dps = mp_dps
    targets = dict(
        alpha = Decimal(str(1/mp.mpf("137.035999206"))),
        pi = Decimal(str(mp.pi)),
        e = Decimal(str(mp.e)),
        gamma = Decimal(str(mp.euler)),
        sqrt2 = Decimal(str(mp.sqrt(2))),
        phi = Decimal(str((1+mp.sqrt(5))/2)),
        sqrt_phi = Decimal(str(mp.sqrt((1+mp.sqrt(5))/2))),
    )
    return targets

def nearest_targets(x: Decimal, targets: Dict[str, Decimal]) -> Tuple[str, Decimal]:
    best_name, best_dist = None, None
    for name, val in targets.items():
        d = abs(x - val)
        if best_dist is None or d < best_dist:
            best_name, best_dist = name, d
    return best_name, best_dist

# ===============================
# Alpha-bridge scan (uniqueness of 729/1000)
# ===============================

def alpha_bridge_scan(D_prefixes: Dict[int, Decimal],
                      alpha: Decimal,
                      qs: List[int]) -> pd.DataFrame:
    rows = []
    for N, D in D_prefixes.items():
        for q in qs:
            best = (None, None, Decimal("Infinity"))
            for p in range(1, q+1):
                val = (Decimal(p)/Decimal(q))*D
                err = abs(val - alpha)
                if err < best[2]:
                    best = (p, q, err)
            p, q, err = best
            rows.append(dict(N=N, p=p, q=q, err=str(err)))
        # also check your p/q = 729/1000
        special = abs((Decimal(729)/Decimal(1000))*D - alpha)
        rows.append(dict(N=N, p=729, q=1000, err=str(special), special="true"))
    return pd.DataFrame(rows)

# ===============================
# Tilt / Mix / Stride
# ===============================

def tms_metrics_for_x(x: Decimal, bases: List[int], depths: List[int]) -> pd.DataFrame:
    recs = []
    for b in bases:
        # compute max depth once, slice
        maxN = max(depths)
        digs = base_digits_decimal(x, b, maxN)
        for N in depths:
            ds = digs[:N]
            c = Counter(ds)
            counts = [c.get(d,0) for d in range(b)]
            freqs = [cnt/N for cnt in counts]
            Hn = entropy_norm_from_counts(counts, b)
            L = longest_run(ds)
            p0 = 1.0/b
            ci = ci95_halfwidth_uniform(b, N)
            max_abs_tilt = max(abs(fr - p0) for fr in freqs)
            rec = dict(base=b, N=N,
                       entropy_norm=Hn,
                       longest_run=L,
                       max_abs_tilt=max_abs_tilt,
                       ci95_halfwidth=ci)
            for d in range(b):
                rec[f"freq_{d}"] = freqs[d]
                rec[f"count_{d}"] = counts[d]
            recs.append(rec)
    return pd.DataFrame(recs)

# ===============================
# MAIN
# ===============================

def main():
    print(f"[{now()}] Start. Building sources…")
    ensure_dir(CFG["OUTDIR"])
    random.seed(42)

    # Build the Fibonacci bigit (decimal embedding)
    M = CFG["BIGIT_DECIMAL_DIGITS"]
    print(f"  Fibonacci word (decimal digits): {M} digits")
    Fbits = fib_word(M)
    DF = embed_decimal_bigit(Fbits)  # Decimal bigit

    # Optionally: control sources
    ctrl = {}
    if CFG["DO_CONTROLS"]:
        print("  Controls: random seeds + Thue–Morse")
        ctrl["thue_morse"] = embed_decimal_bigit(thue_morse(M))
        for i in range(CFG["CONTROL_RANDOM_SEEDS"]):
            ctrl[f"random_{i+1}"] = embed_decimal_bigit(random_01(M, p=0.5))

    # Targets
    targets = build_targets(mp_dps=CFG["MP_DPS"])
    alpha = targets["alpha"]

    # Test 1 — Alpha bridge with prefixes
    print(f"[{now()}] Test 1: alpha-bridge scan")
    D_prefixes = {}
    for N in CFG["PREFIX_NS"]:
        D_prefixes[N] = embed_decimal_bigit(Fbits[:N])
    df_ab = alpha_bridge_scan(D_prefixes, alpha, CFG["SCAN_QS"])
    df_ab.to_csv(os.path.join(CFG["OUTDIR"], "alpha_bridge_scan.csv"), index=False)
    print("  → wrote alpha_bridge_scan.csv")
    # quick print
    print(df_ab.sort_values(["N","err"]).groupby("N").head(3).to_string(index=False))

    # Test 2 — Automaton cycles & nearest constants + phase (mod 3)
    print(f"[{now()}] Test 2: automaton cycles")
    auto_rows = []
    for cyc in CFG["CYCLES"]:
        xs = run_cycle(DF, cyc, steps=CFG["AUTOMATON_STEPS"], keep_frac=True)
        for t,x in enumerate(xs):
            name, dist = nearest_targets(x, targets)
            auto_rows.append(dict(cycle=">".join(cyc), step=t, x=str(+x), nearest=name, dist=str(dist), phase=t%3))
    df_auto = pd.DataFrame(auto_rows)
    df_auto.to_csv(os.path.join(CFG["OUTDIR"], "automaton_hits.csv"), index=False)
    # quick: best hits per constant
    print("  Best per constant (first cycle):")
    print(df_auto.sort_values("dist", key=lambda s: s.map(Decimal)).groupby("nearest").head(1)[["cycle","step","phase","nearest","dist"]].to_string(index=False))

    # Test 3 — Tilt/Mix/Stride across bases and inversion tiers
    print(f"[{now()}] Test 3: tilt/mix/stride (bases & inversion tiers)")
    tms_frames = []
    for k in CFG["INVERSION_TIERS"]:
        xk = apply_inversion_tiers(DF, k)
        df_tms = tms_metrics_for_x(xk, CFG["BASES"], CFG["DEPTHS"])
        df_tms["source"] = "DF"
        df_tms["inversion_tier"] = k
        tms_frames.append(df_tms)

    # Controls TMS
    if CFG["DO_CONTROLS"]:
        for name, x in ctrl.items():
            for k in CFG["INVERSION_TIERS"]:
                xk = apply_inversion_tiers(x, k)
                df_tms_c = tms_metrics_for_x(xk, CFG["BASES"], CFG["DEPTHS"])
                df_tms_c["source"] = name
                df_tms_c["inversion_tier"] = k
                tms_frames.append(df_tms_c)

    df_tms_all = pd.concat(tms_frames, ignore_index=True)
    df_tms_all.to_csv(os.path.join(CFG["OUTDIR"], "tms_metrics.csv"), index=False)
    print("  → wrote tms_metrics.csv")

    # A tiny readable summary: does tilt shrink below CI with N?
    def flag_shrink(g):
        # for a group (base,source,inversion_tier), see if max_abs_tilt drops below CI by N=max
        g = g.sort_values("N")
        last = g.iloc[-1]
        return pd.Series(dict(
            entropy_mean = g["entropy_norm"].mean(),
            longest_run_max = g["longest_run"].max(),
            tilt_last = last["max_abs_tilt"],
            ci_last = last["ci95_halfwidth"],
            tilt_below_CI = bool(last["max_abs_tilt"] <= last["ci95_halfwidth"])
        ))
    summ = df_tms_all.groupby(["source","inversion_tier","base"]).apply(flag_shrink).reset_index()
    summ.to_csv(os.path.join(CFG["OUTDIR"], "tms_shrink_summary.csv"), index=False)
    print("  → wrote tms_shrink_summary.csv")
    print(summ.head(12).to_string(index=False))

    print(f"[{now()}] Done. Files in {CFG['OUTDIR']}:")
    for fn in ["alpha_bridge_scan.csv","automaton_hits.csv","tms_metrics.csv","tms_shrink_summary.csv"]:
        print(" ", fn)

# Run
if __name__ == "__main__":
    main()


[2025-10-03 18:23:12] Start. Building sources…
  Fibonacci word (decimal digits): 8000 digits
  Controls: random seeds + Thue–Morse
[2025-10-03 18:23:13] Test 1: alpha-bridge scan
  → wrote alpha_bridge_scan.csv
   N   p   q                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

/tmp/ipython-input-2170061200.py:354: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  summ = df_tms_all.groupby(["source","inversion_tier","base"]).apply(flag_shrink).reset_index()
